In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
     |████████████████████████████████| 27.4 MB 67.9 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch
import random

In [ ]:
from logging import getLogger

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger
from recbole.trainer import Trainer
from recbole.quick_start import run_recbole, load_data_and_model

from recbole.model.general_recommender import BPR, Pop, NGCF, ItemKNN
from recbole.model.sequential_recommender import GRU4Rec,GCSAN



## GCSAN

In [ ]:
!python --version

Python 3.7.13


In [ ]:
path='/content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/'

#### Prediction

In [ ]:
# /content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/saved/BPR-May-28-2022_16-17-08.pth

# /saved/GCSAN-May-31-2022_09-02-57zz.pth

# model_path_change 

config,model,dataset,train_data,valid_data,test_data=load_data_and_model(model_file=path+'/saved/GCSAN-May-31-2022_09-02-57.pth')

31 May 15:18    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 28
state = INFO
reproducibility = True
data_path = /content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/data/recsys
checkpoint_dir = /content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/saved/
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 8
train_batch_size = 4096
learner = adam
learning_rate = 0.001
neg_sampling = None
eval_step = 1
stopping_step = 3
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [100]
valid_metric = MRR@100
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Param

In [ ]:
from recbole.utils.case_study import full_sort_topk

external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:] # 'user_id' ## interaction history 

In [ ]:
len(external_user_ids)

60000

In [ ]:
external_user_ids

array(['26', '200', '205', ..., '4439694', '4439757', '4439847'],
      dtype='<U7')

In [ ]:
test_data.dataset.item_num




23692

In [ ]:
dataset

recsys
The number of users: 60001
Average actions of users: 4.06885
The number of items: 23692
Average actions of items: 17.767903930131006
The number of inters: 244131
The sparsity of the dataset: 99.98282634565692%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'item_id_list', 'timestamp_list', 'item_length']

In [ ]:
test_data.dataset

recsys
The number of users: 60001
Average actions of users: 1.0
The number of items: 23692
Average actions of items: 4.4921281979195955
The number of inters: 31957
The sparsity of the dataset: 99.99775195091225%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'item_id_list', 'timestamp_list', 'item_length']

In [ ]:
def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))


from recbole.data.interaction import Interaction

In [ ]:
## define predict_num


def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index] ## interaction 
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 100)[1] ## return item_idx


In [ ]:
topk_items = []
for external_user_id in external_user_ids[1:]:
    #_, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=100, device=config['device'])
    topk_iid_list = predict_for_all_item(external_user_id, dataset, model)
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)

print(len(topk_items))

/usr/local/lib/python3.7/dist-packages/recbole/model/sequential_recommender/gcsan.py:187: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  A = torch.FloatTensor(A).to(self.device)


In [ ]:
topk_items = []
for external_user_id in external_user_ids: ## predict 
    #_, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=100, device=config['device'])
    topk_iid_list = predict_for_all_item(external_user_id, dataset, model)
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)

print(len(topk_items))

60000


In [ ]:
len(topk_items[0])

100

In [ ]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['user_id'])
result['prediction'] = external_item_str
result.head()

,user_id,prediction
0,26,19185 3260 17231 23789 21624 16117 27273 6129 ...
1,200,27613 4130 18657 4758 8060 10163 1368 17089 23...
2,205,8194 28128 107 3825 7400 6491 6487 22352 9046 ...
3,495,26942 6853 24020 6491 1515 2790 6916 20028 175...
4,521,26471 12141 5375 17826 8406 742 19270 26085 20...


In [ ]:
## LeaderBoard


test_leaderboard_sessions=pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/data/recsys/test_leaderboard_sessions.csv')

## type change & unique
result = result[result['user_id'].astype(str).isin(list(test_leaderboard_sessions['session_id'].astype(str).unique()))]

In [ ]:
result

,user_id,prediction
0,26,19185 3260 17231 23789 21624 16117 27273 6129 ...
1,200,27613 4130 18657 4758 8060 10163 1368 17089 23...
2,205,8194 28128 107 3825 7400 6491 6487 22352 9046 ...
3,495,26942 6853 24020 6491 1515 2790 6916 20028 175...
4,521,26471 12141 5375 17826 8406 742 19270 26085 20...
...,...,...
59993,4439446,20770 15862 14392 14725 17376 15501 18723 2350...
59994,4439458,26499 7154 17722 20028 22653 23375 6491 20571 ...
59995,4439550,6491 9046 2915 20028 7455 19150 12958 10997 17...
59996,4439653,27442 12179 16479 2769 25955 19912 3402 15262 ...


In [ ]:
res = pd.concat([result, result['prediction'].str.split(' ', expand=True)], axis=1).drop(columns=['prediction']).set_index('user_id').stack().reset_index()

In [ ]:
res

,user_id,level_1,0
0,26,0,19185
1,26,1,3260
2,26,2,17231
3,26,3,23789
4,26,4,21624
...,...,...,...
4999995,4439757,95,22665
4999996,4439757,96,16157
4999997,4439757,97,24984
4999998,4439757,98,697


In [ ]:
res = res.rename(columns={'user_id':'session_id', 'level_1':'rank', 0:'item_id'})
res = res[['session_id','item_id','rank']] ## column change
res['rank'] = res['rank'] + 1

In [ ]:
res

,session_id,item_id,rank
0,26,19185,1
1,26,3260,2
2,26,17231,3
3,26,23789,4
4,26,21624,5
...,...,...,...
4999995,4439757,22665,96
4999996,4439757,16157,97
4999997,4439757,24984,98
4999998,4439757,697,99


In [ ]:
res.to_csv('/content/drive/MyDrive/Colab_Notebooks/대학원3학기/Recsys_challenge/submission/submission_gcsan_test.csv', index=False)